In [ ]:
import pandas as pd
from IPython.core.pylabtools import figsize

dataIn = './../dataIn/'
dataOut = './../dataOut/'

In [ ]:
filename = dataIn + 'payment07.csv'
payment = pd.read_csv(filename, encoding='CP949')
payment

In [ ]:
payment['성별'].unique()

In [ ]:
# 성별에 따른 교통비 집계
mygrouping = payment.groupby('성별')['교통비']
mygrouping.sum()

In [ ]:
mygrouping.count()

In [ ]:
result = payment.groupby('성별')['출장기간'].agg('mean')
result

In [ ]:
agg_dict = {'출장기간':'sum', '출장지역':'count'}
result = payment.groupby('성별').agg(agg_dict)
result

In [ ]:
# 출장지역별 성별의 교통비의 총합
result = payment.groupby(['출장지역', '성별'])['교통비'].agg('sum')
result

In [ ]:
mydict = {'교통비':['sum', 'mean', 'max'], '출장기간':['mean', 'min']}
result = payment.groupby(['출장지역', '성별']).agg(mydict)
result

In [ ]:
filename = dataIn + 'welfareCleanNew.csv'
welfare = pd.read_csv(filename, encoding='UTF-8')
welfare.columns

In [ ]:
print('# 각 컬럼에 대한 데이터 분포 확인')
for col in welfare.columns:
    print(f'{col} 컬럼 정보')
    print(welfare[col].unique())
    print('-'*30)
# end for

In [ ]:
# 결혼 유무별 소득의 평균 그래프

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family = 'Malgun Gothic')

In [ ]:
welfare['결혼 유무'].unique()

In [ ]:
chardata = welfare.groupby('결혼 유무')['소득'].mean()
chardata

In [ ]:
chardata.plot(kind = 'pie', figsize=(7,5), autopct='%1.1f%%')
plt.title('결혼 유무별 소득의 평균', size=17)
plt.axis('equal')
plt.legend(labels=chardata.index, loc='lower right')

In [ ]:
# 소득과 나이의 산점도
welfare.plot(kind='scatter', figsize=(10, 5), x='소득', y='나이')
plt.title('산점도 : 소득 ds 나이')

In [ ]:
# 결혼 유무별 소득의 박스 플롯
marriage = [item for item in welfare['결혼 유무'].unique()]
marriage # 범주형 데이터

xData = [] # 범주형 테이터별 소득 정보를 담아 놓은 리스트
for mar in marriage:
    xData.append(welfare[welfare['결혼 유무']==mar]['소득'])
# end for

# xData

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(15, 5))

plt.boxplot(
    x=[
        welfare[welfare['결혼 유무']=='결혼']['소득'],
        welfare[welfare['결혼 유무']=='이혼']['소득'],
        welfare[welfare['결혼 유무']=='무응답']['소득']
    ],
    tick_labels=['결혼', '이혼', '무응답'],
    vert=False
)

plt.title('결혼 유무별 소득(박스 플롯)')

In [ ]:
concern = ['성별', '지역구', '나이', '소득'] # 분석 대상 컬럼
newWelfare = welfare[concern]
newWelfare.info()

In [ ]:
newWelfare.head() # head() 함수는 5행만 보여줌 ↔ tail()

In [ ]:
print('# 소득 컬럼 누락 데이터 확인')
if newWelfare['소득'].hasnans :
    print('누락 데이터 있음')
else:
    print('누락 데이터 없음')

In [ ]:
print(f'# 소득 컬럼의 형상 : {newWelfare.shape}')

In [ ]:
print('# 소득 컬럼 기초 통계량 확인')
# 메소드 체이닝 : 메소드 작성시 dot(.)를 연속으로 작성하는 기법
newWelfare['소득'].describe().to_frame().T # 소득 데이터를 시리즈로 나열하여 데이터 프레임을 전치하여 한줄에 데이터가 나오게끔 처리

In [ ]:
print('# 소득 오름차순으로 정렬해보기') # 데이터베이스의 orderBy 구문과 맥락이 비슷함
newWelfare['소득'].sort_values().head(10).to_frame().T

In [ ]:
print('# 소득 내림차순으로 정렬해보기')
newWelfare['소득'].sort_values(ascending=False).head(10).to_frame().T

In [ ]:
# 소득이 1200 보다 큰 데이터들은 이상치(outlier)라고 가정하고 삭제합니다.
print(f'# 제거 전 개수 : {len(newWelfare)}')
newWelfare = newWelfare.loc[newWelfare['소득'] <= 1200]
print(f'# 제거 후 개수 : {len(newWelfare)}')

print(f'# 소득 최대값 : {newWelfare["소득"].max()}')
print(f'# 소득 최소값 : {newWelfare["소득"].min()}')

In [ ]:
# 히스토그램 그리기 (히스토그램은 연속형 데이터에만 적용)

In [ ]:
hist_chart = newWelfare["소득"]
hist_chart.plot(kind='hist', figsize=(10, 5), bins=50)
plt.title('소득 Histogram', size=17)
plt.xlabel('소득')

In [ ]:
# 데이터 이산화(데이터의 형태 변환)

In [ ]:
import numpy as np
# np.inf :
mybins = [-np.inf, 200, 500, 700, 1000, np.inf] # 나눌 구간 지정
cut_line = pd.cut(newWelfare["소득"], bins=mybins)
cut_line.head().to_frame().T

In [ ]:
cut_line.value_counts().to_frame()

In [ ]:
print('# 이산화된 소득별로 성별 비율 확인')
# observed=False 이면 실제로 존재하지 않는 범주 데이터도 같이 보여줍니다.
result = 100.0 * newWelfare.groupby(cut_line, observed=False)['성별'].value_counts(normalize=True).round(4)
result

In [ ]:
newWelfare.head()

In [ ]:
print('# 이산화 값을 한글로 변경 후 파생 컬럼으로 추가하기')
hangul_label = ['저소득', '중저소득', '중소득', '중고소득', '고소득']
cuts_label = pd.cut(newWelfare["소득"], bins=mybins, labels=hangul_label)
# cuts_label.head()

print('# 파생 컬럼으로 추가하기')
newWelfare.loc[:, '범주형소득'] = cuts_label
newWelfare.head()

In [ ]:
print('# 성별, 범주형 소득별 소득 평균')
chartdata = newWelfare.groupby(['성별', '범주형소득'], observed=False)['소득'].mean().to_frame()
chartdata = chartdata.reset_index()
chartdata

In [ ]:
print('성별 컬럼은 행에, 범주형소득 컬럼은 열에, 값을 소득 정보를 배치하세요.')
chartdata = chartdata.pivot(index='성별', columns='범주형소득', values='소득')
chartdata

In [ ]:
chartdata.plot(kind='barh', figsize=(15, 8))
plt.title('성별/범주형소득별 소득의 평균', size=17)
plt.xlabel('소득 평균', size=15)
plt.ylabel('성별', size=15)
plt.legend(loc='best', fontsize=12) # loc='best'는 배치하기 좋은데 자동으로 위치시킵니다.